<a href="https://colab.research.google.com/github/xd-mau5/radar/blob/main/Creacion_de_Imagenes_de_Radar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install boto3==1.28.15
%pip install botocore
%pip install Cartopy==0.21.1
%pip install imageio==2.31.1
%pip install matplotlib==3.7.2
%pip install pandas==1.5.3
%pip install arm-pyart

In [2]:
import imageio
import pyart
import os
import shutil
import timeit
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import datetime
import pandas as pd
import glob
import matplotlib.pyplot as plt


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
def get_date_utc():
    """
    Returns the current date in UTC timezone as a string in the format 'YYYY/MM/DD'.
    """
    fechaDeHoy = datetime.datetime.now(tz=datetime.timezone.utc)
    fechaDeHoy = pd.to_datetime(fechaDeHoy, format='%Y/%m/%d')
    fechaDeHoy = fechaDeHoy.strftime('%Y/%m/%d')
    return fechaDeHoy

In [4]:
def get_file_list_from_s3(date, radar):
    """
    Returns a list of objects in the S3 bucket 's3-radaresideam' that exceed 1 MB in size, and are located in the
    directory 'l2_data/date/radar', where 'date' is a string in the format 'YYYY/MM/DD', and 'radar' is a string
    representing the name of the radar.

    Parameters:
    date (str): A string representing the date in the format 'YYYY/MM/DD'.
    radar (str): A string representing the name of the radar.

    Returns:
    list: A list of objects in the S3 bucket 's3-radaresideam' that exceed 1 MB in size, and are located in the
    directory 'l2_data/date/radar'.
    """
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    bucket = 's3-radaresideam'
    s3_prefix = 'l2_data/' + date + '/' + radar
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket, Prefix=s3_prefix)
    # Crear una lista de objetos en el bucket de S3 que superen 1 MB de tamaño
    listaDeObjetos = []
    for page in pages:
        for obj in page['Contents']:
            if obj['Size'] > 900000:
                listaDeObjetos.append(obj['Key'])
    # Ordenar la lista de objetos
    listaDeObjetos.sort()
    return listaDeObjetos

In [5]:
def download_files_from_s3(file_list, folder):
    """
    The function `download_files_from_s3` downloads a list of files from an S3 bucket and saves them in
    a specified folder.

    :param file_list: The `file_list` parameter is a list of file paths in the S3 bucket that you want
    to download. Each file path should be a string
    :param folder: The `folder` parameter is the directory where you want to download the files from S3.
    It is the destination folder where the downloaded files will be stored
    """
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.mkdir(folder)
    for file in file_list:
        s3.download_file('s3-radaresideam', file, folder + '/' + file.split('/')[-1])
        print('Descargando ' + file.split('/')[-1])
    print('Descarga finalizada')

In [6]:
import glob
import os

def get_file_list_from_folder(folder):
    """
    Returns a list of files in a folder that exceed 1 MB in size.

    Parameters:
    folder (str): A string representing the path to the folder.

    Returns:
    list: A list of files in the folder that exceed 1 MB in size.
    """
    listaDeArchivos = glob.glob(folder + '/*')
    listaDeArchivos = [archivo for archivo in listaDeArchivos if os.path.getsize(archivo) > 900000]
    listaDeArchivos.sort()
    return listaDeArchivos

In [7]:
def get_location_from_radar(file):
    """
    Returns the latitude and longitude of the radar.

    Parameters:
    file (str): A string representing the path to the radar file.

    Returns:
    tuple: A tuple containing the latitude and longitude of the radar.
    """
    radar_lat = file.latitude['data'][0]
    radar_lon = file.longitude['data'][0]
    return radar_lat, radar_lon

In [8]:
def get_date_from_radar_to_colombian_time(file):
    """
    The function `get_date_from_radar_to_colombian_time` receives a radar file path and returns the date and time
    of the radar in Colombian time (UTC-5).

    :param file: The `file` parameter is a string representing the path to the radar file.
    :return: The function returns a string representing the date and time of the radar in Colombian time (UTC-5).
    """
    radar = pyart.io.read(file)
    fecha = radar.time['units'][14:]
    fecha = datetime.datetime.strptime(fecha, '%Y-%m-%dT%H:%M:%SZ')
    fecha = fecha - datetime.timedelta(hours=5) # Fix: subtract timedelta from datetime object
    fecha = fecha.strftime('%Y/%m/%d %H:%M:%S')
    return fecha

In [9]:
def get_range_from_radar(file):
    """
    Returns the maximum range of the radar in kilometers.

    Parameters:
    file (str): A string representing the path to the radar file.

    Returns:
    float: The maximum range of the radar in kilometers.
    """
    radar = pyart.io.read(file)
    return radar.range['data'][-1]/1000

In [10]:
def get_radar_name(file):
    """
    Returns the name of the radar.

    Parameters:
    file (str): A string representing the path to the radar file.

    Returns:
    str: The name of the radar.
    """
    name = file.metadata['instrument_name']
    return name

In [11]:
def create_plot(data, var, vmin, vmax, cities_dict):
    """
    The function `create_plot` receives radar data, a variable, minimum and maximum values, and a dictionary of cities
    with their respective latitude and longitude coordinates. It creates a plot of the radar data with the specified
    variable and range, and adds features such as city markers, radar location marker, and range rings. The plot is saved
    as a PNG file with the date and time of the radar data as the filename.

    :param data: The `data` parameter is a pyart radar object representing the radar data.
    :param var: The `var` parameter is a string representing the variable to be plotted.
    :param vmin: The `vmin` parameter is a float representing the minimum value of the colorbar.
    :param vmax: The `vmax` parameter is a float representing the maximum value of the colorbar.
    :param cities_dict: The `cities_dict` parameter is a dictionary representing the cities to be plotted on the map,
                        with their respective latitude and longitude coordinates.

    :return: The function returns a string representing the filename of the saved PNG file.
    """
    display = pyart.graph.RadarMapDisplay(data)
    fig = plt.figure(figsize=(12, 10), dpi=100)
    fecha = data.time['units'][14:]
    fecha = datetime.datetime.strptime(fecha, '%Y-%m-%dT%H:%M:%SZ')
    fecha = fecha - datetime.timedelta(hours=5) # Fix: subtract timedelta from datetime object
    fecha = fecha.strftime('%Y/%m/%d %H:%M:%S')
    ax = plt.axes(projection=ccrs.PlateCarree())
    # Hacer que el mapa sea proyectado en toda el área de la imagen
    ax.set_position([0, 0, 1, 1]) # type: ignore
    # Agregar características del mapa
    ax.add_feature(cfeature.BORDERS, linewidth=0.5) # type: ignore
    ax.add_feature(cfeature.STATES, linewidth=0.5) # type: ignore
    ax.add_feature(cfeature.OCEAN, linewidth=0.5, color='lightgray') # type: ignore
    # Definir el área de visualización del mapa utilizando las coordenadas del radar de Corozal
    ax.set_extent([-79, -72, 6, 12]) # type: ignore
    # Agregar líneas de latitud y longitud, solamente del lado izquierdo y abajo
    ax.gridlines(color='gray', linestyle='--', draw_labels=True) # type: ignore
    # Cargar los datos geoespaciales de los municipios
    cities_lat = [city['lat'] for city in cities_dict.values()]
    cities_lon = [city['lon'] for city in cities_dict.values()]
    # Graficar los marcadores de los municipios
    ax.plot(cities_lon, cities_lat, 'ko', markersize=3, transform=ccrs.PlateCarree())
    for city in cities_dict.keys():
        ax.text(cities_dict[city]['lon'], cities_dict[city]['lat'], city, transform=ccrs.PlateCarree(), fontsize=10)
    # Agregar el marcador del radar en el mapa
    radar_lat, radar_lon = get_location_from_radar(data)
    ax.plot(radar_lon, radar_lat, 'k*', markersize=8, transform=ccrs.PlateCarree())
    # Guardar la imagen con el nombre de la fecha del radar
    display.plot_ppi_map(
        var,
        0,
        vmin=vmin,
        vmax=vmax,
        title = 'Radar Corozal - {} UTC-5'.format(fecha),
        resolution='50m',
        cmap='pyart_NWSRef',
        colorbar_label='Factor de Reflectividad (dBZ)',
        colorbar_orient='vertical',
        fig=fig,
        filter_transitions=True,
        ax=ax,
        projection=ccrs.PlateCarree(),
        raster=True
        )
    # Sombrear el área fuera del rango de alcance del radar, llenar con color gris
    display.plot_range_ring(300, 1000, ax=ax, color='k', ls='--', alpha=1)
    # Todo lo que esté fuera del rango de alcance del radar, llenar con color gris
    fig.tight_layout(pad=0, w_pad=0, h_pad=0, rect=(0, 0, 1, 1))
    fecha = fecha.replace('/', '_')
    plt.savefig('{}.png'.format(fecha))
    plt.close()
    # Guardar la imagen con la fecha del radar
    print("Imagen creada con nombre", '{}.png'.format(fecha))
    return '{}.png'.format(fecha)

In [12]:
def create_gif_from_images(image_list, radar):
    """
    Creates a gif file from a list of images.

    Parameters:
    image_list (list): A list of image file paths.
    radar (str): The name of the radar.

    Returns:
    None
    """
    # Generar el gif
    gif_file = radar + '.gif'
    with imageio.get_writer(gif_file, mode='I', duration=200) as writer:
        for image in image_list:
            image_data = imageio.v2.imread(image)
            writer.append_data(image_data) # type: ignore
    print('Gif creado con nombre', radar + '.gif')

In [13]:
def delete_files_from_folder(folder):
    """
    Deletes all files in a folder and removes the folder itself.

    Parameters:
    folder (str): The name of the folder to be deleted.

    Returns:
    None
    """
    shutil.rmtree(folder)
    print('Carpeta', folder, 'eliminada')
    images = glob.glob('*.png')
    for image in images:
        os.remove(image)
    print('Imagenes eliminadas')

In [14]:
def main():
    """
    This function is the main function that runs the radar program. It downloads radar data from an S3 bucket, creates a list of files, creates a list of images, creates a dictionary of municipalities, creates a gif file from the images, and deletes the downloaded files.

    Parameters:
    None

    Returns:
    None
    """
    # Definir la fecha de hoy
    fechaDeHoy = get_date_utc()
    # Definir el radar
    radar = 'Corozal'
    # Definir el folder donde se van a descargar los archivos
    folder = 'Corozal'
    # Descargar los archivos del bucket de S3
    print('Lista de archivos en S3')
    lista_s3 = get_file_list_from_s3(fechaDeHoy, radar)
    print(lista_s3)
    download_files_from_s3(get_file_list_from_s3(fechaDeHoy, radar), folder)
    # Crear una lista de archivos en el folder
    listaDeArchivos = get_file_list_from_folder(folder)
    print(listaDeArchivos)
    # Crear una lista de imágenes
    listaDeImagenes = []
    print(get_range_from_radar(listaDeArchivos[0]))
    # Crear un diccionario de municipios
    municipios_dict = {'Monteria': {'lat': 8.76, 'lon': -75.885555555556},
                'Sincelejo': {'lat': 9.29, 'lon': -75.39},
                'Cartagena': {'lat': 10.39, 'lon': -75.48},
                'Barranquilla': {'lat': 10.96, 'lon': -74.80},
                'Santa Marta': {'lat': 11.24, 'lon': -74.21},
                'San Antero': {'lat': 9.37, 'lon': -75.76},
                'Cerete': {'lat': 8.89, 'lon': -75.80},
                'Chinu': {'lat': 9.1097222222222, 'lon': -75.398055555556}
                }
    # Crear una lista de imágenes usando multiprocessing
    for file in listaDeArchivos:
        radar_data = pyart.io.read(file)
        listaDeImagenes.append(create_plot(radar_data, 'reflectivity', 10, 80, municipios_dict))

    # Crear el gif
    create_gif_from_images(listaDeImagenes, radar)
    # Eliminar los archivos del folder
    delete_files_from_folder(folder)

In [15]:
if __name__ == '__main__':
    print(timeit.timeit("main()", setup="from __main__ import main", number=1), 'segundos')

Lista de archivos en S3
['l2_data/2023/08/13/Corozal/COR230813000003.RAWCHLK', 'l2_data/2023/08/13/Corozal/COR230813000502.RAWCHM8', 'l2_data/2023/08/13/Corozal/COR230813001003.RAWCHMY', 'l2_data/2023/08/13/Corozal/COR230813001503.RAWCHNL', 'l2_data/2023/08/13/Corozal/COR230813002003.RAWCHPA', 'l2_data/2023/08/13/Corozal/COR230813002503.RAWCHPZ', 'l2_data/2023/08/13/Corozal/COR230813003003.RAWCHRN', 'l2_data/2023/08/13/Corozal/COR230813003503.RAWCHSB', 'l2_data/2023/08/13/Corozal/COR230813004002.RAWCHT1', 'l2_data/2023/08/13/Corozal/COR230813004503.RAWCHTP', 'l2_data/2023/08/13/Corozal/COR230813005002.RAWCHUD', 'l2_data/2023/08/13/Corozal/COR230813005503.RAWCHV2', 'l2_data/2023/08/13/Corozal/COR230813010003.RAWCHVS', 'l2_data/2023/08/13/Corozal/COR230813010503.RAWCHWE', 'l2_data/2023/08/13/Corozal/COR230813011002.RAWCHX4', 'l2_data/2023/08/13/Corozal/COR230813011503.RAWCHXT', 'l2_data/2023/08/13/Corozal/COR230813012003.RAWCHYG', 'l2_data/2023/08/13/Corozal/COR230813012502.RAWCHZ5', 'l2

/usr/local/lib/python3.10/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_ocean.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.10/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_0_boundary_lines_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.10/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_1_states_provinces_lakes.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.10/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/50m_physical/ne_50m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.10/dist-packages/cartopy/io/__init__.p

Imagen creada con nombre 2023_08_12 19:00:03.png
Imagen creada con nombre 2023_08_12 19:05:02.png
Imagen creada con nombre 2023_08_12 19:10:03.png
Imagen creada con nombre 2023_08_12 19:15:03.png
Imagen creada con nombre 2023_08_12 19:20:03.png
Imagen creada con nombre 2023_08_12 19:25:03.png
Imagen creada con nombre 2023_08_12 19:30:03.png
Imagen creada con nombre 2023_08_12 19:35:03.png
Imagen creada con nombre 2023_08_12 19:40:02.png
Imagen creada con nombre 2023_08_12 19:45:03.png
Imagen creada con nombre 2023_08_12 19:50:02.png
Imagen creada con nombre 2023_08_12 19:55:03.png
Imagen creada con nombre 2023_08_12 20:00:03.png
Imagen creada con nombre 2023_08_12 20:05:03.png
Imagen creada con nombre 2023_08_12 20:10:02.png
Imagen creada con nombre 2023_08_12 20:15:03.png
Imagen creada con nombre 2023_08_12 20:20:03.png
Imagen creada con nombre 2023_08_12 20:25:02.png
Imagen creada con nombre 2023_08_12 20:30:03.png
Imagen creada con nombre 2023_08_12 20:35:02.png
Imagen creada con no